<h2>Импорт библиотек</h2>
<p>Spark, функции для работы с данными, а также matplotlib, pandas и seaborn для визуализаций.
Инициализируется <code>findspark</code> для корректного запуска Spark из Python.</p>
<hr>

In [ ]:
import glob
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when, lit, unix_timestamp, countDistinct, avg, stddev, exp, explode, datediff, current_date, min as spark_min
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
import os
findspark.init("/your_path")

<h2>Создание SparkSession</h2>
<p>Создание <code>SparkSession</code> с увеличенным лимитом памяти для драйвера и исполнителей.
Также задаётся уровень логов <code>ERROR</code> и включается опция игнорирования повреждённых файлов.</p>
<hr>

In [ ]:
spark = SparkSession.builder \
    .appName("OzonApparelAnalysis") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")

<h2>Загрузка parquet-файлов</h2>
<p>Сбор путей ко входным parquet-файлам:
<code>orders</code> (заказы),
<code>tracker</code> (действия пользователей),
<code>items</code> (товары),
<code>categories</code> (иерархия категорий),
<code>participants</code> (тестовая выборка).</p>
<hr>

In [ ]:
items_files = glob.glob('/srv/data/ml_ozon_recsys_train_final_apparel_items_data/*.parquet')
categories_tree = glob.glob('/srv/data/ml_ozon_recsys_train_final_categories_tree/*.parquet')
participants_files = glob.glob('/srv/data/ml_ozon_recsys_test_for_participants/*.parquet')
orders_files = glob.glob('/srv/data/preprocessed/orders_preprocessed/*.parquet')
tracker_files = glob.glob('/srv/data/preprocessed/tracker_preprocessed/*.parquet')
test_files = glob.glob('/srv/data/ml_ozon_recsys_test/*.parquet')

<h2>Чтение parquet-файлов</h2>
<p>Чтение parquet-файлов в Spark DataFrame: <code>orders</code>, <code>tracker</code>,
<code>items</code>, <code>categories</code>. Каждый датафрейм соответствует отдельному источнику данных.</p>
<hr>

In [ ]:
orders = spark.read.parquet(*orders_files)
tracker = spark.read.parquet(*tracker_files)
items = spark.read.parquet(*items_files)
categories = spark.read.parquet(*categories_tree)

<h2>Подсчёт строк</h2>
<p>Подсчёт количества строк в каждой загруженной таблице: <code>orders</code>, <code>tracker</code>,
<code>items</code>, <code>categories</code>.</p>
<hr>

In [ ]:
print(f"В orders {orders.count()} строк")
print(f"В tracker {tracker.count()} строк")
print(f"В items {items.count()} строк")
print(f"В categories {categories.count()} строк")

<h2>Анализ таблицы orders</h2>
<p><b>Анализ заказов</b>: проверка пропусков, распределение статусов заказов, динамика по датам,
топ-пользователи и товары, распределение длительности выполнения заказа.</p>

<p>Проверка заказов на пропуски: считаем количество <code>null</code> значений для каждой колонки.</p>
<hr>

In [ ]:
null_counts = orders.select([
    spark_sum(col(c).isNull().cast("int")).alias(c) for c in orders.columns
])

null_counts.show()

<h2>Визуализации по заказам</h2>
<p>Распределение статусов, динамика заказов по датам, топ-10 пользователей, топ-10 товаров
и распределение времени выполнения заказа.</p>
<hr>

In [ ]:
# ======================
# 1. Распределение статусов заказов
# ======================
status_counts = orders.groupBy("last_status").count().toPandas()

plt.figure(figsize=(8,5))
plt.bar(status_counts["last_status"], status_counts["count"])
plt.title("Распределение статусов заказов")
plt.ylabel("Количество")
plt.xticks(rotation=30)
plt.show()


# ======================
# 2. Кол-во заказов по датам
# ======================
orders_by_date = orders.groupBy("created_date").count().orderBy("created_date").toPandas()

plt.figure(figsize=(14,6))
plt.plot(orders_by_date["created_date"], orders_by_date["count"], marker="o")
plt.title("Распределение заказов по датам")
plt.ylabel("Количество заказов")
step = 7
plt.xticks(orders_by_date["created_date"][::step], rotation=45)

plt.show()


# ======================
# 3. Топ-10 пользователей
# ======================
top_users = orders.groupBy("user_id").count().orderBy(col("count").desc()).limit(10).toPandas()

plt.figure(figsize=(10,5))
plt.bar(top_users["user_id"].astype(str), top_users["count"])
plt.title("ТОП-10 пользователей по заказам")
plt.ylabel("Количество заказов")
plt.xticks(rotation=45)
plt.show()


# ======================
# 4. Топ-10 товаров
# ======================
top_items = orders.groupBy("item_id").count().orderBy(col("count").desc()).limit(10).toPandas()

plt.figure(figsize=(10,5))
plt.bar(top_items["item_id"].astype(str), top_items["count"])
plt.title("ТОП-10 товаров по заказам")
plt.ylabel("Количество заказов")
plt.xticks(rotation=45)
plt.show()


# ======================
# 5. Время выполнения заказа (delivery duration)
# ======================
orders_with_duration = orders.withColumn(
    "duration_days",
    (unix_timestamp("last_status_timestamp") - unix_timestamp("created_timestamp")) / 86400
)

duration_stats = orders_with_duration.select("duration_days").toPandas()

plt.figure(figsize=(10,5))
plt.hist(duration_stats["duration_days"], bins=50, edgecolor="k")
plt.title("Распределение времени выполнения заказа (в днях)")
plt.xlabel("Длительность (дни)")
plt.ylabel("Частота")
plt.show()


<h2><b>Добавление затухающего веса заказам</b></h2>

Мы хотим учесть не только факт заказа, но и его **свежесть** и **статус**.  

Для доставленных заказов:
$$
w = e^{-0.01 \cdot \big( \text{current\_date} - \text{created\_date} \big)}
$$

Для заказов в статусе *processed_orders*:
$$
w = 0.3 \cdot e^{-0.01 \cdot \big( \text{current\_date} - \text{created\_date} \big)}
$$

Для всех остальных:
$$
w = 0
$$

---
**Интерпретация:**
- новые доставленные заказы → вес близок к 1  
- старые → вес стремится к 0  
- `processed_orders` учитываются, но слабее (×0.3)  
- остальные статусы игнорируются

In [ ]:
orders_with_decay = orders.withColumn(
    "order_weight",
    when(col("last_status") == "delivered_orders",
         exp(-0.01 * datediff(current_date(), col("created_date"))))
    .when(col("last_status") == "processed_orders",
          0.3 * exp(-0.01 * datediff(current_date(), col("created_date"))))
    .otherwise(0.0)
)
orders_with_decay.show(5, truncate=False)

<h2>Сохранение таблицы orders</h2>
<p>Сохранение датафрейма с рассчитанными весами заказов в parquet-файл.</p>
<hr>

In [ ]:
# orders_with_decay.write.mode("overwrite").parquet("/your_path")

<h2>Дополнительный анализ orders</h2>
<p>Длительность выполнения, базовые статистики, доля заказов по статусам,
средняя длительность по статусам, распределение числа заказов по дням и корреляция числовых признаков.</p>
<hr>

In [ ]:
# ======================
# 1. Создадим колонку "длительность заказа"
# ======================
orders_with_duration = orders.withColumn(
    "duration_days",
    (unix_timestamp("last_status_timestamp") - unix_timestamp("created_timestamp")) / 86400
)

# ======================
# 2. Базовые статистики
# ======================
print("=== Базовые статистики по duration_days ===")
orders_with_duration.select(
    F.min("duration_days").alias("min_days"),
    F.max("duration_days").alias("max_days"),
    F.mean("duration_days").alias("mean_days"),
    F.stddev("duration_days").alias("std_days")
).show()

print("=== Уникальные пользователи и товары ===")
orders.select(
    countDistinct("user_id").alias("unique_users"),
    countDistinct("item_id").alias("unique_items")
).show()

print("=== Среднее кол-во заказов на пользователя ===")
orders.groupBy("user_id").count().select(avg("count")).show()

# ======================
# 3. Доля заказов по статусам
# ======================
status_share = (
    orders.groupBy("last_status")
    .count()
    .withColumn("share", (col("count") / orders.count()) * 100)
    .toPandas()
)
print(status_share)

plt.figure(figsize=(8,5))
sns.barplot(data=status_share, x="last_status", y="share")
plt.title("Доля заказов по статусам (%)")
plt.xticks(rotation=30)
plt.show()

# ======================
# 4. Duration vs Status (среднее время в зависимости от статуса)
# ======================
duration_by_status = (
    orders_with_duration.groupBy("last_status")
    .agg(F.mean("duration_days").alias("mean_duration"))
    .toPandas()
)

plt.figure(figsize=(8,5))
sns.barplot(data=duration_by_status, x="last_status", y="mean_duration")
plt.title("Средняя длительность заказа по статусам")
plt.xticks(rotation=30)
plt.ylabel("Средняя длительность (дни)")
plt.show()

# ======================
# 5. Статистика по датам
# ======================
orders_by_date = orders.groupBy("created_date").count().toPandas()

print("Среднее число заказов в день:", orders_by_date["count"].mean())
print("Стандартное отклонение заказов в день:", orders_by_date["count"].std())

plt.figure(figsize=(12,6))
sns.histplot(orders_by_date["count"], bins=30, kde=True)
plt.title("Распределение числа заказов по дням")
plt.xlabel("Количество заказов в день")
plt.ylabel("Частота")
plt.show()

# ======================
# 6. Корреляция
# ======================
min_date = orders.select(spark_min("created_date")).collect()[0][0]

orders_num = orders_with_decay \
    .withColumn("created_ts_num", unix_timestamp("created_timestamp")) \
    .withColumn("last_status_ts_num", unix_timestamp("last_status_timestamp")) \
    .withColumn("created_date_num", datediff(col("created_date"), lit(min_date)))

numeric_cols = ["created_ts_num", "last_status_ts_num", "created_date_num", "order_weight"]

orders_pd = orders_num.select(numeric_cols).toPandas()

plt.figure(figsize=(7,6))
sns.heatmap(orders_pd.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Корреляция числовых признаков (orders)")
plt.show()


<h2>Анализ таблицы tracker</h2>
<p>Таблица <code>tracker</code> содержит информацию о действиях пользователей с товарами:
<code>item_id</code>, <code>user_id</code>, <code>timestamp</code>, <code>action_type</code>, <code>action_widget</code>, <code>date</code>.</p>

<p>Проверка на пропуски в tracker: подсчёт количества <code>null</code> значений по каждому столбцу.</p>
<hr>

In [ ]:
spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")

null_counts = tracker.select([
    spark_sum(col(c).isNull().cast("int")).alias(c) for c in tracker.columns
])
null_counts.show(truncate=False)

<h2>Визуализации по tracker</h2>
<p>Распределение типов действий, топ-10 источников <code>action_widget</code>,
активность пользователей по датам, топ-10 пользователей и товаров по активности.</p>
<hr>

In [ ]:
# 1. Распределение типов действий
action_type_dist = tracker.groupBy("action_type").count().toPandas()

plt.figure(figsize=(8,5))
sns.barplot(data=action_type_dist, x="action_type", y="count")
plt.title("Распределение типов действий (action_type)")
plt.xticks(rotation=45)
plt.ylabel("Количество")
plt.show()

# 2. ТОП-10 источников action_widget
action_widget_top = tracker.groupBy("action_widget").count().orderBy("count", ascending=False).limit(10).toPandas()

plt.figure(figsize=(10,5))
sns.barplot(data=action_widget_top, x="action_widget", y="count")
plt.title("ТОП-10 источников (action_widget)")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Количество")
plt.show()


# 3. Активность пользователей по датам
activity_by_date = tracker.groupBy("date").count().orderBy("date").toPandas()

plt.figure(figsize=(15,5))
plt.plot(activity_by_date["date"], activity_by_date["count"], marker="o")
plt.title("Активность пользователей по датам")
plt.xlabel("Дата")
plt.ylabel("Количество действий")

# шаг для меток (например, раз в 7 дней)
step = 7
plt.xticks(activity_by_date["date"][::step], rotation=45)

plt.show()


# 4. ТОП-10 пользователей по активности
top_users = tracker.groupBy("user_id").count().orderBy("count", ascending=False).limit(10).toPandas()

plt.figure(figsize=(10,5))
sns.barplot(data=top_users, x="user_id", y="count")
plt.title("ТОП-10 пользователей по количеству действий")
plt.xticks(rotation=45)
plt.ylabel("Количество действий")
plt.show()


# 5. ТОП-10 товаров по активности
top_items = tracker.groupBy("item_id").count().orderBy("count", ascending=False).limit(10).toPandas()

plt.figure(figsize=(10,5))
sns.barplot(data=top_items, x="item_id", y="count")
plt.title("ТОП-10 товаров по количеству действий")
plt.xticks(rotation=45)
plt.ylabel("Количество действий")
plt.show()


<h2>Очистка tracker</h2>
<p>Заполнение пропусков в колонке <code>action_widget</code> самым популярным значением.</p>
<hr>

In [ ]:
# 1. Найдём самое популярное значение
top_widget = (
    tracker.groupBy("action_widget")
    .count()
    .orderBy(col("count").desc())
    .first()[0]
)

print("Самый популярный action_widget:", top_widget)

# 2. Заполним пропуски этим значением
tracker_filled = tracker.withColumn(
    "action_widget",
    when(col("action_widget").isNull(), lit(top_widget)).otherwise(col("action_widget"))
)

# 3. Проверим результат
tracker_filled.select("action_widget").distinct().show(10, truncate=False)

In [ ]:
# tracker_filled.write.mode("overwrite").parquet("/your_path")

<h2>Анализ таблицы items</h2>
<p>Проверка пропусков, анализ распределений по <code>catalogid</code>, варианты на модель и категорию,
а также распределение размерности эмбеддингов CLIP.</p>
<hr>

In [ ]:
null_counts = items.select([
    spark_sum(col(c).isNull().cast("int")).alias(c) for c in items.columns
])
null_counts.show(truncate=False)

<h2>Визуализации по items</h2>
<p>Анализ таблицы <code>items</code> с помощью визуализаций:</p>
<ul>
  <li><b>Топ-20 категорий по количеству товаров</b> — строится распределение по <code>catalogid</code>.</li>
  <li><b>Количество вариантов на модель</b> — гистограмма распределения числа товаров на <code>model_id</code>.</li>
  <li><b>Количество вариантов на категорию</b> — гистограмма распределения товаров внутри <code>catalogid</code>.</li>
  <li><b>Длина вектора эмбеддингов</b> — проверка размерности признаков <code>fclip_embed</code> для всех товаров.</li>
</ul>
<hr>

In [ ]:
# ======================
# 1. Распределение catalogid
# ======================
catalog_dist = items.groupBy("catalogid").count().orderBy(col("count").desc()).toPandas()

plt.figure(figsize=(12,6))
sns.barplot(data=catalog_dist.head(20), x="catalogid", y="count")
plt.title("Топ-20 категорий по количеству товаров (catalogid)")
plt.xticks(rotation=45)
plt.ylabel("Количество товаров")
plt.show()


# ======================
# 2. Кол-во вариантов на модель
# ======================
variants_per_model = items.groupBy("model_id").count().toPandas()

plt.figure(figsize=(10,5))
plt.hist(variants_per_model["count"], bins=50, edgecolor="k")
plt.title("Распределение количества вариантов на модель")
plt.xlabel("Количество вариантов")
plt.ylabel("Частота")
plt.show()


# ======================
# 3. Кол-во вариантов на catalogid
# ======================
variants_per_catalog = items.groupBy("catalogid").count().toPandas()

plt.figure(figsize=(10,5))
plt.hist(variants_per_catalog["count"], bins=50, edgecolor="k")
plt.title("Распределение количества товаров внутри категорий (catalogid)")
plt.xlabel("Количество товаров")
plt.ylabel("Частота")
plt.show()


# ======================
# 4. Длина вектора fclip_embed
# ======================
items_embed_len = items.select(F.size(col("fclip_embed")).alias("embed_len")).toPandas()

plt.figure(figsize=(8,5))
plt.hist(items_embed_len["embed_len"], bins=20, edgecolor="k")
plt.title("Распределение длины векторов fclip_embed")
plt.xlabel("Размерность вектора")
plt.ylabel("Частота")
plt.show()
